### instalação de pacotes [executar]

In [ ]:
!pip install pdfminer.six
!pip3 install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.6 MB 4.6 MB/s 
     |████████████████████████████████| 4.0 MB 38.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports [executar]

In [ ]:
import os
import unidecode
import re
from pdfminer.high_level import extract_text
from glob import glob
from datetime import datetime
import pandas as pd
import numpy as np

# Explorando os PDFs
- adaptando código para leitura de todos pdfs

In [ ]:
cct_com_problemas = '/content/drive/MyDrive/Hacka datathONS/CCTs com problemas'

### métodos para extrair e limpar texto dos PDFs

In [ ]:
def extract_from_pdf(file_path) -> str:
  # TODO: tem parte não sendo extraídas
  # exemplo pdf 9 (com problemas) n extra se ribeiro gonçalves, legenda de tabela.
  try:
    return extract_text(file_path)
  except ValueError:
    return file_path
    
def to_ascii(text):
  '''
  padroniza o texto [...]
  '''
  outstring = ''
  for i in range(len(text)):
    outstring = outstring + unidecode.unidecode(text[i])
  return outstring

def clean_text(text):
  '''
  transforma em lowercase e remove quebras de linha e outros caracteres
  '''
  text = str.lower(text)
  #pattern=r'[^a-zA-z0-9\s]'
  #text=re.sub(pattern,'',text)
  text=re.sub('_', '', text)
  text=re.sub('\n', '', text)
  text=re.sub(' +', ' ', text)
  return text

### teste

In [ ]:
filenames = os.listdir(cct_com_problemas)
text = extract_from_pdf(os.path.join(cct_com_problemas, filenames[2]))
text = to_ascii(text)
text = clean_text(text)

In [ ]:
for it in filenames:
  text = extract_from_pdf(os.path.join(cct_com_problemas, it))
  text = to_ascii(text)
  text = clean_text(text)
  print(extract_parecer(text))


2021-pa-0044

2021-pa-0151
2020-pa-0209
2020-pa-0073
2021-pa-0141
2021-pa-0164
2021-pa-0118
2020-pa-0071



2019-pa-0242

2021-pa-0054


2021-pa-0127
2020-pa-0070


In [ ]:
text

'contrato de conexao ao sistema de transmissao cct 081/2021-00 contratantes: companhia hidro eletrica do sao francisco - chesf serra talhada i energia spe ltda.- ufv zimba envolvere i interveniencia: operador nacional do sistema - ons instalacao envolvida: subestacao se bom nome - bno \x0ccontrato de conexao ao sistema de transmissao - cct cct 081/2021-00 contrato de conexao ao sistema de transmissao que entre si fazem a chesf e a ufv zimba envolvere i, com a interveniencia do ons, acesso compartilhado na se bom nome. correspondente ao de um lado e doravante denominada simplesmente transmissora, a companhia hidro eletrica do sao francisco - chesf, empresa concessionaria de servicos publicos de energia eletrica, mediante primeiro termo aditivo contrato de concessao de servico publico de transmissao de energia eletrica no 061/2001 - ctt 061/2001, firmado com a aneel em 04 de dezembro de 2012, com sede na cidade de recife, estado de pernambuco, na rua delmiro gouveia, no 333, san martin, 

___
# Validações - Grupo 3

1. Existe menção ao Parecer de Acesso no contrato?
  - possibilidades:
    - busca simples pelo termo "Parecer de Acesso" ou abreviação, caso exista.

2. Verificar data de assinatura do parecer de acesso (na base)
  - possibilidades:
    - buscar o padrão do Parecer ex.: '2020-PA-0030' 
    - buscar esse parecer na base
      - não existe: alerta 1
      - existe
        - pega a data e retorna
        - calcula os dias
          - mais de 90: alerta 2
          - entre 60 e 90: alerta 3

3. Ponto de conexão contratado
  - O ponto de conexão com a Rede Básica do contrato bate com o da base para este Parecer de Acesso?
___
Alertas:
- parecer vencido (mais de 90 dias)
- mais de 60 dias e menos de 30 dias para vencer
- não existe parecer na base

In [ ]:
def mencao_parecer(doc: str) -> bool:
  pa = re.findall('parecer de acesso', text)
  return True if pa else False

def extract_parecer(doc: str) -> str:
  '''
  extrai do documento o código do Parecer de Acesso de acordo com um padrão regex
  '''
  parecer_pattern = '[0-9]{4}[-]pa[-][0-9]{4}'
  parecer = np.unique(re.findall(parecer_pattern, doc))
  if parecer.size:
    return parecer[0]
  else:
    return ''

def has_parecer(parecer: str) -> bool:
  '''
  verifica se o parecer extraído do documento está cadastrado na base
  '''
  return base.documento_emitido.str.contains(parecer).any()
  
def get_parecer_date(parecer: str):
  '''
  recupera da base a data do parecer que está no documento
  '''
  return base.loc[base.documento_emitido.str.contains('2020-PA-0030'), 'data_emissao'].values[0]

def get_ponto_conexao(parecer: str) -> str:
  ponto = base.loc[base.documento_emitido.str.contains(parecer), 'pontoconexao'].values
  if ponto.size:
    return ponto[0]
  else:
    return ''

def get_all_ponto_conexao():
  pontos = base.pontoconexao.dropna().unique()
  pontos = [to_ascii(it) for it in pontos]
  pontos = [clean_text(it) for it in pontos]
  return pontos

def validate_item11(doc):
  return mencao_parecer(doc)
  
def validate_item12(doc):
  pa = extract_parecer(doc)
  if has_parecer(pa):
    date = get_parecer_date(pa)
    return (datetime.now().date() - pd.to_datetime(date).date()).days
  else:
    return False

def validate_item13(doc):
  pa = extract_parecer(doc)
  if has_parecer(pa):
    pc = get_ponto_conexao(pa)
    if pc in doc:
      return True
    else:
      return False
  
base = pd.read_excel('/content/drive/MyDrive/Hacka datathONS/DatathONS/Parecer de acesso.xlsx')
#base.data_emissao = base.data_emissao.apply(lambda x: pd.to_datetime(x))

In [ ]:
item11 = mencao_parecer(text)
#item12 tem os dias ou False, se não tiver o parecer na base
item12 = validate_item12(text)
#item13 True se bater o ponto de conexão, False se forem diferentes
if item12 >= 90:
  print(f'parecer de acesso vencido: {item12} dias.')
elif item12 >= 60 & item12 < 90:
  print(f'parecer de acesso perto de vencer: faltam {90 - item12} dias.')

validate_item13(text)


parecer de acesso perto de vencer: faltam 90 dias.
